# Code for a painless q-learning tutorial

- Refer to <http://mnemstudio.org/path-finding-q-learning-tutorial.htm>
- Build a simple AI based on `numpy`.
- Write this just for study
- Author: [JasonQSY](https://github.com/JasonQSY)

In [1]:
# we just need numpy
import numpy as np

In [2]:
# initial
# q is the tabular representation for q func.
q = np.matrix(np.zeros([6, 6]))

# r is the tabular representation for rewards.
# r is predefined and remains unchanged.
r = np.matrix([[-1, -1, -1, -1,  0,  -1], 
               [-1, -1, -1,  0, -1, 100], 
               [-1, -1, -1,  0, -1,  -1], 
               [-1,  0,  0, -1,  0,  -1], 
               [ 0, -1, -1,  0, -1, 100], 
               [-1,  0, -1, -1,  0, 100]])

# hyperparameter
gamma = 0.8
epsilon = 0.4

There is a difference between the code here and the guide. Here we use epsilon greedy, i.e. with probability epsilon we choose a random action, otherwise we choose the greedy action to maximize local Q value.

In [3]:
# the main training loop
for episode in range(101):
    # random initial state
    state = np.random.randint(0, 6)

    while (state != 5): # stop only when state == TERMINAL
        # Filter feasible actions.
        # Even in random case, we cannot choose actions whose r[state, action] = -1.
        # It's not about reinforcement learning. These actions are not feasible physically.
        possible_actions = []
        possible_q = []
        for action in range(6):
            if r[state, action] >= 0:
                possible_actions.append(action)
                possible_q.append(q[state, action])
                
        # Step next state, here we use epsilon-greedy algorithm.
        action = -1
        if np.random.random() < epsilon:
            # choose random action
            action = possible_actions[np.random.randint(0, len(possible_actions))]
        else:
            # greedy
            action = possible_actions[np.argmax(possible_q)]
        
        # Update Q value
        q[state, action] = r[state, action] + gamma * q[action].max()
        
        # Go to the next state
        state = action
        
    # Display training progress
    if episode % 10 == 0:
        print("------------------------------------------------")
        print("Training episode: %d" % episode)
        print(q)

------------------------------------------------
Training episode: 0
[[   0.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.  100.]
 [   0.    0.    0.    0.    0.    0.]]
------------------------------------------------
Training episode: 10
[[   0.    0.    0.    0.   80.    0.]
 [   0.    0.    0.   64.    0.  100.]
 [   0.    0.    0.   64.    0.    0.]
 [   0.   80.    0.    0.    0.    0.]
 [   0.    0.    0.   64.    0.  100.]
 [   0.    0.    0.    0.    0.    0.]]
------------------------------------------------
Training episode: 20
[[   0.     0.     0.     0.    80.     0. ]
 [   0.     0.     0.    64.     0.   100. ]
 [   0.     0.     0.    64.     0.     0. ]
 [   0.    80.    51.2    0.    80.     0. ]
 [  64.     0.     0.    64.     0.   100. ]
 [   0.     0.     0.     0.     0.     0. ]]
------------------------------------------------
T

We can see that the Q matrix converges and it is the same as the matrix shown in the guide. The only difference is that all the entries are 0 for row 6. That's because we do not update the q matrix when state == 5. I have no idea why the row 6 in the guide has postive values. Anyway, let's verify the result.

In [4]:
# verify
for i in range(10):
    # one episode
    print("episode: %d" % i) 
    
    # random initial state
    state = np.random.randint(0, 6)
    print("the robot borns in " + str(state) + ".")
    for _ in range(20): # prevent endless loop
        if state == 5:
            break
        action = np.argmax(q[state])
        print("the robot goes to %d." % action)
        state = action

episode: 0
the robot borns in 4.
the robot goes to 5.
episode: 1
the robot borns in 2.
the robot goes to 3.
the robot goes to 1.
the robot goes to 5.
episode: 2
the robot borns in 5.
episode: 3
the robot borns in 0.
the robot goes to 4.
the robot goes to 5.
episode: 4
the robot borns in 0.
the robot goes to 4.
the robot goes to 5.
episode: 5
the robot borns in 0.
the robot goes to 4.
the robot goes to 5.
episode: 6
the robot borns in 0.
the robot goes to 4.
the robot goes to 5.
episode: 7
the robot borns in 1.
the robot goes to 5.
episode: 8
the robot borns in 2.
the robot goes to 3.
the robot goes to 1.
the robot goes to 5.
episode: 9
the robot borns in 4.
the robot goes to 5.


Now the robot shows his ability to find the correct path rapidly.